<div class="knitr source">
    <h1><p style="text-align:left; font-size: 24px"> Exploración de análisis de datos, para los tweets de COVID 19</p></h1>
    <h2><p style="text-align:left; font-size: 20px"> UCSP</p></h2>
  
</div>

<img src="https://www.paho.org/sites/default/files/styles/flexslider_full/public/2020-03/blue-covid-banner.jpg?h=96546727&itok=cZemcbKa" align = 'center'>
<p><i style="color:red;">Presentado por: Bhernard S. Beisaga Arenas </i></p>


![](http://)<p style="text-align:justify; font-size: 18px">El siguiente trabajo explora los Tweets que han sido publicados en la pandemia mundial de COVID 19. Mostraremos visualmente la percepcion de la gente basado en los datos. Usaremos algunos gráficos  que se han enseñado en el curso de DIPLOMADO</p>

<p style="text-align:justify; font-size: 18px">
Link de consulta en Kaggle: <a href='https://www.kaggle.com/bbeisaga/ucsp-covid19-assessment'>https://www.kaggle.com/bbeisaga/ucsp-covid19-assessment</a>
    </p>


<div style="color:white;
           display:fill;
           border-radius:80px;
           background-color:#5e2129;
           font-size:200%;
           font-family:Verdana;
           letter-spacing:2px">

<p style="padding: 10px 40px;
              color:white;
          text-align:left"> CONTENIDO    
 
</p>
</div>

<p style="text-align:justify; font-size: 18px">
    <ul style="text-align:justify; font-size: 18px">
        <li>Conjunto de datos</li>
        <li>Visualización</li>
        <li>Analisis de tweets</li>
        <li>Visualización Geoespacial</li>
</ul>
    </p>
    

<div style="color:white;
           display:fill;
           border-radius:80px;
           background-color:#5e2129;
           font-size:200%;
           font-family:Verdana;
           letter-spacing:2px">

<p style="padding: 10px 40px;
              color:white;
          text-align:left"> CONJUNTO DE DATOS****    
 
</p>
</div>

In [ ]:
# librerias requeridas para leeer el csv y añadir algun comportamiento

import numpy as np
import pandas as pd
import plotly.express as px
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib import rcParams
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from iso3166 import countries
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot


rcParams['figure.figsize'] = 10,7.5
rcParams['figure.dpi'] = 80



<p style="text-align:justify; font-size: 18px">
    Visualizando el conjunto de datos para el respectivo analisis.
</p>

In [ ]:
df = pd.read_csv('/kaggle/input/covid19-tweets/covid19_tweets.csv')
df.head(15)

In [ ]:
df.info()

<p style="text-align:justify; font-size: 18px">
    Como se puede observar, en las columnas de la tabla existe una gran cantidad de valores en blanco NaN, lo cual podria darnos algunos inconvenientes en el análisis de datos
    </p>
    

<div style="color:white;
           display:fill;
           border-radius:80px;
           background-color:#5e2129;
           font-size:200%;
           font-family:Verdana;
           letter-spacing:2px">

<p style="padding: 10px 40px;
              color:white;
          text-align:left"> VISUALIZACION DE LOS DATOS
 
</p>
</div>

<p style="text-align:justify; font-size: 18px">
    En el siguiente gráfico muestra el número de Twees por usuario
    </p>
    

In [ ]:
numero_tweets = df['user_name'].value_counts().reset_index()
numero_tweets.columns = ['user_name', 'tweets']


sns.barplot(x = "tweets", y = "user_name", data = numero_tweets.head(10))
plt.show()

<p style="text-align:justify; font-size: 18px">El siguiente gráfico muestas las cuentas con la mayor cantidad de seguidores y que han realziado Tweets de COVID19.
    </p>

In [ ]:
top_users = df.sort_values('user_followers', ascending =  False).drop_duplicates(subset = 'user_name', keep = 'first')
top_users = top_users[['user_name', 'user_followers']]
top_users = pd.merge(top_users, numero_tweets, 'inner')


#Normalize the scale to make the color bar on the right of the bar plot
norm = plt.Normalize(top_users['tweets'].min(), top_users['tweets'].max())
sm = plt.cm.ScalarMappable(cmap="Blues_r", norm=norm)
sm.set_array([])
#Show the barplot with color bar
ax = sns.barplot(x="user_followers", y = "user_name", data = top_users.head(20), hue = 'tweets', dodge = False, palette = 'Blues_r')
ax.get_legend().remove()
ax.figure.colorbar(sm)
plt.show()


<p style="text-align:justify; font-size: 18px">
    Según el gráfico China Xinhua Newa ha publicado muchos Twess sobre COVID 19, pero no tiene tantos seguyidores como CNN.
    </p>
    <p style="text-align:justify; font-size: 18px">
Observemos los dispositivos usados para los Twees de COVID 19. Podemos concluir que la mayoria proviene de alguna App probablemente descargada e instalada en dispositivos Android como celulares
    </p>
    

In [ ]:
device = df['source'].value_counts().reset_index()
device.columns = ['source', 'count']
device['percent_tweets'] = round(device['count']/device['count'].sum()*100, 2)


sns.barplot(x = "percent_tweets", y = "source", data = device.head(10))
plt.show()

<div style="color:white;
           display:fill;
           border-radius:80px;
           background-color:#5e2129;
           font-size:200%;
           font-family:Verdana;
           letter-spacing:2px">

<p style="padding: 10px 40px;
              color:white;
#           text-align:left"> ANALISIS DE TWEETS
 
</p>
</div>

<p style="text-align:justify; font-size: 18px">
    En está seccion analizaremos la informacion de texto dentro de los hashtags que se han utilizado en los twess, tambien remplazaremos los valores NaN por alguna otra cosa
    </p>

In [ ]:
def hashtags_split(x):
    return str(x).lower().replace('[','').replace(']','').replace("'",'').replace(" ", '').split(',')

hashtag_tweets = df.copy()
hashtag_tweets['hashtag'] = hashtag_tweets['hashtags'].apply(lambda row: hashtags_split(row))
hashtag_tweets = hashtag_tweets.explode('hashtag')
hashtag_tweets.loc[hashtag_tweets['hashtag'] == '', 'hashtag'] = 'No Hashtag'
hashtag_tweets.head()

![](http://)<p style="text-align:justify; font-size: 18px">
        En el siguiente gráfico veremos cual es el hashtag mas utilizado por los usuarios
    </p>


In [ ]:
hashtag_number = hashtag_tweets['hashtag'].value_counts().reset_index()
hashtag_number.columns = ['hashtag', 'count']

sns.barplot(x="count", y = "hashtag", data = hashtag_number.head(5))
plt.show()

<p style="text-align:justify; font-size: 18px">
    El siguinte paso será hacer una nube de palabras utilizadas en el Tweets
    </p>

In [ ]:
text = "".join(tweet for tweet in df['text'])
stopwords = set(STOPWORDS)
stopwords.update(['https', 't','co', 'many', 's'])

wordcloud = WordCloud(stopwords=stopwords, background_color='white').generate(text)

plt.imshow(wordcloud)
plt.axis('off')
plt.title('Prevalent words for all tweets')
plt.show()

<div style="color:white;
           display:fill;
           border-radius:80px;
           background-color:#5e2129;
           font-size:200%;
           font-family:Verdana;
           letter-spacing:2px">

<p style="padding: 10px 40px;
              color:white;
#           text-align:left"> VISUALIZACIÓN GEOESPACIAL
 
</p>
</div>

<p style="text-align:justify; font-size: 18px">
    El siguiente gráfico muestra la distribución de Tweest sobre el COVID 19 en el mundo
    </p>

In [ ]:
location = df['user_location'].value_counts().reset_index()
location.columns = ['user_location', 'count']
location = location[location['user_location'] != 'NA']
location = location.sort_values(['count'], ascending = False)

sns.barplot(x="count", y = "user_location", data = location.head(10))
plt.show()

<p style="text-align:justify; font-size: 18px">
    Como se puede obsersar los USA y la India son los que mas públican Tweets
    </p>

In [ ]:
!pip install geopandas


In [ ]:
import pycountry

def alpha3code(column):
    CODE = []
    for country in column:
        try:
            code = pycountry.countries.get(name=country).alpha_3
            CODE.append(code)
        except:
            CODE.append('None')
    return CODE


location['CODE'] = alpha3code(location['user_location'])
location.head()

In [ ]:
import geopandas
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
world.columns = ['pop_est', 'continent', 'name', 'CODE', 'gdp_md_est', 'geometry']
world_merge = pd.merge(world, location, on='CODE')

location_merge = pd.read_csv('../input/countries-latitude-longitude/countries-latitude-longitude.csv')
world_merge = world_merge.merge(location_merge, on='name').sort_values(by='count', ascending=False).reset_index()
world_merge = world_merge[['user_location', 'count', 'latitude','longitude']]
world_merge.head()

In [ ]:
import folium
from folium import plugins
from folium.plugins import HeatMap

folium_map = folium.Map(location=[50,0],
                       zoom_start=3,
                       tiles='CartoDB dark_matter')

world_merge['latitude']=world_merge['latitude'].fillna(0)
world_merge['longitude']=world_merge['longitude'].fillna(0)

plugins.FastMarkerCluster(data=list(zip(world_merge['latitude'].values, world_merge['longitude'].values))).add_to(folium_map)
arr = world_merge[['latitude', 'longitude']].values

HeatMap(arr, radius = 15).add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map

In [ ]:
Eso es todo, muchas por la atención prestada